In [4]:
import numpy as np
import plotly.graph_objects as go

Вариант №15
\begin{equation*}
    \begin{cases}
        u_{xx}+u_{yy}=f(x,y), \quad x\in (0,1), \quad y\in (0,2)\\
        u(x,0)=u_{true}(x,0)\\
        u(x,2)=u_{true}(x,2)\\
        u(0,y)=u_{true}(0,y)\\
        u(1,y)=u_{true}(1,y)\\
    \end{cases}
\end{equation*}

\begin{equation*}
    u_{true}(x,y)=(3x+2y)\sin(5x+y)-3x^2-y^2
\end{equation*}

\begin{equation*}
    f(x,y)=\frac{\partial ^2u_{true}}{\partial x^2}+\frac{\partial ^2u_{true}}{\partial y^2}=-26(3x+2y)\sin(5x+y)+34\cos(5x+y)-8
\end{equation*}

In [5]:
# Define u_true
def u_true(x, y):
    return (3*x+2*y)*np.sin(5*x+y)-3*x**2-y**2

# Define f
def f(x, y):
    return -26*(3*x+2*y)*np.sin(5*x+y)+34*np.cos(5*x+y)-8

In [9]:
# Plot surface u_true with x in [0,1] and y in [0,2]
x = np.linspace(0, 1, 100)
y = np.linspace(0, 2, 100)
X, Y = np.meshgrid(x, y)
Z = u_true(X, Y)
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.show()

In [16]:
# Используя метод минимальных невязок, решить систему уравнений Ax = b

# Константы
x_min = 0
x_max = 1
y_min = 0
y_max = 2
h1 = 0.1
h2 = 0.1
N1 = int((x_max - x_min) / h1)
N2 = int((y_max - y_min) / h2)

# Minimal residual method for solving linear systems
def min_res(A, b, x0, eps):
    D = np.conjugate(A)*A
    x = x0
    w = np.dot(A,x) - b
    tau = np.dot(np.dot(A,w).T, w) / np.dot(np.dot(A,w).T, np.dot(A,w))
    while np.linalg.norm(w) > eps:
        x = x - tau*w
        w = np.dot(A,x) - b
        tau = np.dot(np.dot(A,w).T, w) / np.dot(np.dot(A,w).T, np.dot(A,w))
    return x

# Minumum error method for solving linear systems
def min_error(A, b, x0, eps):
    D = A
    D_inv = np.linalg.inv(D)
    A_conj = np.conjugate(A)
    x = x0
    r = np.dot(A,x) - b
    w = np.dot(D_inv, np.dot(A_conj,r))
    tau = np.dot(r.T, r) / np.dot(r.T, np.dot(A,w))
    while np.linalg.norm(r) > eps:
        x = x - tau*w
        r = np.dot(A,x) - b
        w = np.dot(D_inv, np.dot(A_conj,r))
        tau = np.dot(r.T, r) / np.dot(r.T, np.dot(A,w))
        # print(np.linalg.norm(r))
    return x

# Построить разностную аппроксимацию для уравнения Пуассона на квадрате [a, b] x [c, d] с шагом h1 и h2 и функцией f(x, y)
def poisson(x_min, x_max, y_min, y_max, h1, h2, f):
    N1 = int((x_max - x_min) / h1)
    N2 = int((y_max - y_min) / h2)
    A = np.zeros((N1 * N2, N1 * N2))
    b = np.zeros((N1 * N2, 1))
    for i in range(N1):
        for j in range(N2):
            k = i * N2 + j
            if i == 0 or i == N1 - 1 or j == 0 or j == N2 - 1:
                A[k, k] = 1
                b[k] = u_true(x_min + i * h1, y_min + j * h2)
            else:
                A[k, k] = 2 / h1 ** 2 + 2 / h2 ** 2
                A[k, k - 1] = -1/h1**2
                A[k, k + 1] = -1/h1**2
                A[k, k - N2] = -1/h2**2
                A[k, k + N2] = -1/h2**2
                b[k] = -f(x_min + i * h1, y_min + j * h2)
    return A, b

# Построить матрицу A и вектор b для уравнения Пуассона на квадрате [0, 1] x [0, 2] с шагом h1=0.01 и h2=0.01 и функцией f(x, y)
A, b = poisson(x_min, x_max, y_min, y_max, h1, h2, f)

# A2, b2 = poisson2(x_min, x_max, y_min, y_max, h1, h2, f)

# Решить систему уравнений Ax = b методом минимальных невязок с начальным приближением x0 = 0 и точностью eps = 10^-6
# u = min_res(A, b, np.zeros((N1 * N2, 1)), 10**(-3))
u = min_error(A, b, np.zeros((N1 * N2, 1)), 10**(-3))
# u = np.linalg.solve(A, b)

# Преобразовать вектор x в матрицу U
x = np.linspace(x_min, x_max, N1)
y = np.linspace(y_min, y_max, N2)
X, Y = np.meshgrid(x, y)
U = np.zeros((N1, N2))
for i in range(N1):
    for j in range(N2):
        U[i][j] = u[i * N2 + j]
U = U.transpose()

# Построить график решения
fig = go.Figure(data=[go.Surface(z=U, x=X, y=Y)])
# Изменить границы осей
fig.update_layout(scene=dict( xaxis=dict(range=[x_min, x_max], autorange=False), yaxis=dict(range=[y_min, y_max], autorange=False)))
fig.show()